# *Scan 1D analyses*

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pathlib import Path
from measurements_table import create_measurements_table
import utils
import pandas
import plotly.express as px
import plotly.graph_objects as go

# Load data

In [ ]:
measurements_table_df = create_measurements_table()

# For each of the "scan 1D seeping bias voltage" I collect all the single voltage 1D scans that were created in such "voltage sweep scan".
scans_and_sub_measurements_df = pandas.DataFrame()
for root_measurement_name in measurements_table_df.query('Type=="scan 1D sweeping bias voltage"').index:
    with open(utils.path_to_measurements_directory/Path(root_measurement_name)/Path('scan_1D_sweeping_bias_voltage/README.txt'), 'r') as ifile:
        for idx, line in enumerate(ifile):
            if idx == 0:
                continue
            scans_and_sub_measurements_df = scans_and_sub_measurements_df.append({'Scan name': line.replace('\n',''), 'Root measurement name': root_measurement_name}, ignore_index=True)
scans_and_sub_measurements_df.set_index('Scan name', inplace=True)
display(scans_and_sub_measurements_df)

In [ ]:
display(set(scans_and_sub_measurements_df['Root measurement name']))

Now actually read the measured data:

In [ ]:
measured_data_df = pandas.DataFrame()
for measurement_name in scans_and_sub_measurements_df.index:
    #if scans_and_sub_measurements_df.loc[measurement_name,'Root measurement name'] not in {'20211104233152_#45_sweeping_bias_voltage','20211022095225_#1_sweeping_bias_voltage'}:
    #    continue
    this_measurement_data = utils.read_and_pre_process_1D_scan_data(measurement_name)
    measured_data_df = measured_data_df.append(this_measurement_data)

Add some info related to the device, as well as some values of measurement variables...

In [ ]:
for measurement_name in set(measured_data_df['Measurement name']):
    this_measurement_rows = measured_data_df['Measurement name']==measurement_name
    measured_data_df.loc[this_measurement_rows, 'Set bias voltage (V)'] = float(measurements_table_df.loc[measurement_name, 'Bias voltage (V)'])
    measured_data_df.loc[this_measurement_rows, 'Laser DAC'] = measurements_table_df.loc[measurement_name, 'Laser DAC']
    this_measurement_device_name = list(set(this_measurement_data['Device']))[0]
    for param in {'Trench depth','Trenches','Trench process','Pixel border','Contact type','Wafer'}:
        measured_data_df.loc[this_measurement_rows, param] = utils.get_devices_specs_dictionary(this_measurement_device_name)[param.lower()]

In [ ]:
display(measured_data_df)
print(sorted(measured_data_df.columns))

From now on we should not touch `measured_data_df` anymore, just read it.

# Measured variables vs distance and comparison with geometry

In [ ]:
averaged_data_per_position_df = utils.mean_std(
    measured_data_df,
    by = list(set(measured_data_df.columns).difference({
        'Amplitude (V)',
        'Bias current (A)',
        'Bias voltage (V)',
        'Collected charge (V s)',
        'Laser DAC',
        'Noise (V)',
        'Normalized collected charge',
        'Rise time (s)',
        'Time over noise (s)',
        'When',
        'n_trigger',
        't_10 (s)', 't_20 (s)', 't_30 (s)', 't_40 (s)', 't_50 (s)', 't_60 (s)', 't_70 (s)', 't_80 (s)', 't_90 (s)', 
        'x (m)', 'y (m)', 'z (m)',
    }))
)
display(averaged_data_per_position_df)
print(sorted(averaged_data_per_position_df.columns))

In [ ]:
figs_to_draw_geometry = []
for y in {'Normalized collected charge','Collected charge (V s)','Amplitude (V)'}:
    fig = utils.line(
        data_frame = averaged_data_per_position_df.query('n_pulse==1'),
        line_group = 'Measurement name',
        x = 'Distance - offset (m)',
        y = y + ' mean',
        error_y = y + ' std',
        error_y_mode = 'bands',
        color = 'Device',
        title = str(y),
        hover_name = 'Measurement name',
    )
    figs_to_draw_geometry.append(fig)
for idx,fig in enumerate(figs_to_draw_geometry):
    for x in [-125e-6, 0, 125e-6]:
        fig.add_shape(type="line",
            yref = "paper",
            x0 = x,
            y0 = 0,
            x1 = x,
            y1 = 1,
            line = dict(
                color = 'black',
                dash = 'dash',
            ),
        )
    fig.show()

## Inter-pixel distance plot

In [ ]:
df = pandas.DataFrame()
df = df.append(interpixel_distances_df)
measurements_table_persistent_df = pandas.read_excel('measurements_table_persistent.ods', engine='odf', index_col='Measurement name')
df['All pads DC grounded?'] = measurements_table_persistent_df['All pads DC grounded?']
df = df[df['Root measurement name'].isin({
    '20211103181351_#14_sweeping_bias_voltage',
    '20211028161049_#34_sweeping_bias_voltage',
    '20211029103905_#34_sweeping_bias_voltage',
    '20211029201114_#36_sweeping_bias_voltage',
    '20211031200029_#4_sweeping_bias_voltage',
    '20211104233152_#45_sweeping_bias_voltage',
    '20211101203924_#52_sweeping_bias_voltage',
    '20211025184544_#65_sweeping_bias_voltage',
    '20211030201504_#68_sweeping_bias_voltage',
    '20211102143313_#84_sweeping_bias_voltage',
})]
df['Wafer'] = df['Wafer'].astype(int)

fig = utils.line(
    data_frame = df.sort_values(['Device specs','Device','Set bias voltage (V)']).reset_index(),
    x = 'Set bias voltage (V)',
    y = 'Inter-pixel distance (m)',
    color = 'Wafer',
    symbol = 'Pixel border',
    line_dash = 'Contact type',
    hover_name = 'Measurement name',
    markers = True,
    title = 'Inter-pixel distance vs bias voltage using normalized collected charge with linear interpolation',
    grouped_legend = True,
    line_group = 'Root measurement name',
    labels = {
        'Set bias voltage (V)': 'Bias voltage (V)',
    },
    text = 'Trenches',
)
fig.show()

In [ ]:
fig.write_html(f'inter-pixel_distance_vs_bias_voltage.html', include_plotlyjs='cdn')